In [7]:
import requests

In [8]:
import itertools
import os

# Ok I originally thought I'd be parsing the whole tree, but I've since decided to
# only parse the author thread. My code is thus a bit overkill, but. eh.
tweet_ids = {"1327608557441851392"}
database = {}
batch_size = 100
author_id = "144817179"

raw_database = {"tweets": {}, "media": {}, "users": {}, "tweet_refs": {}}

In [9]:
def process_batch(batch_ids):
    url = "https://api.twitter.com/2/tweets"
    headers = {"Authorization": f"Bearer {os.environ.get('BEARER_TOKEN')}"}
    params = {
        "ids": ",".join(batch_ids),
        "tweet.fields": "attachments,text,created_at,author_id,entities,referenced_tweets",
        "expansions": "attachments.media_keys,referenced_tweets.id,entities.mentions.username",
        "media.fields": "url,preview_image_url",
        "user.fields": "profile_image_url,url,username"
    }
    response = requests.get(url, headers=headers, params=params).json()
    tweets = [t for t in response["data"] if t["author_id"] == author_id]
    includes = response.get("includes", {})
    raw_database["tweets"] |= {t["id"]: t for t in tweets}
    if "media" in includes:
        raw_database["media"] |= {m["media_key"]: m for m in includes["media"]}
    if "users" in includes:
        raw_database["users"] |= {m["username"]: m for m in includes["users"]}
    if "tweets" in includes:
        raw_database["tweet_refs"] |= {m["id"]: m for m in includes["tweets"] if m["author_id"] == author_id}

In [10]:
tweet_ids -= set(raw_database["tweets"])

while tweet_ids:
    batch_ids = set(itertools.islice(tweet_ids, batch_size))
    process_batch(batch_ids)
    tweet_ids -= batch_ids
    tweet_ids |= {tweet["id"] for tweet in raw_database["tweet_refs"].values()} - {tweet["id"] for tweet in raw_database["tweets"].values()}
    print(tweet_ids)

{'1327175178514788353'}
{'1326815195373694976'}
{'1326812164590628871'}
{'1326651920300314626'}
{'1326650321771376640'}
{'1326648865701900289'}
{'1326648529109078016'}
{'1326472282022289409'}
{'1326468088108359689'}
{'1326467205140242433'}
{'1326465987005329410'}
{'1326464352174690304'}
{'1326463843036491777'}
{'1318839439477645314'}
{'1315989809647759360'}
{'1315988470037450752'}
{'1313538343855108096'}
{'1307977052759494656'}
{'1277973321213075466'}
{'1277973318952353792'}
{'1277938842490847232'}
{'1277937533259862017'}
{'1277935844083544070'}
{'1277930359079030788'}
{'1277917133163450371'}
{'1277917128486772737'}
{'1277915538111553539'}
{'1277912733565009920'}
{'1277912731098775553'}
{'1277912728330534912'}
{'1277912724509466630'}
{'1277912722638876672'}
set()


In [94]:
raw_database

{'tweets': {'1277912724509466630': {'author_id': '144817179',
   'referenced_tweets': [{'type': 'replied_to', 'id': '1277912722638876672'}],
   'id': '1277912724509466630',
   'attachments': {'media_keys': ['3_1277910702351618049']},
   'entities': {'mentions': [{'start': 0,
      'end': 13,
      'username': 'marieboiseau'}],
    'urls': [{'start': 18,
      'end': 41,
      'url': 'https://t.co/IRBYVOGxF3',
      'expanded_url': 'https://www.marieboiseau.com/shop',
      'display_url': 'marieboiseau.com/shop',
      'status': 200,
      'title': 'Shop — Marie Boiseau',
      'description': 'Quelques informations concernant le shop : -Je ne peux pas donner de délai de livraison précis car je gère tout toute seule sur mon temps libre : les impressions, les préparations de commandes, la gestion des mails, l’envoi à la Poste, etc. Je fais ça en plus de mon métier d’illustratrice. Merc',
      'unwound_url': 'https://www.marieboiseau.com/shop'},
     {'start': 42,
      'end': 65,
      '

In [12]:
import json, pathlib

In [13]:
pathlib.Path("../data/database.json").write_text(json.dumps(raw_database))

89402

In [120]:
def get_recent_tweet_ids(session: requests.Session) -> set:
    url = "https://api.twitter.com/2/tweets/search/recent"
    params = {"query": f"from:garagedeloffre conversation_id:1277912722638876672"}
    tweets = []
    while True:
        response = session.get(url, headers={"Authorization": f"Bearer {os.environ.get('BEARER_TOKEN')}"}, params=params).json()
        tweets.extend(response["data"])
        if "next_tokens" in response["meta"]:
            params["next_tokens"] = response["meta"]["next_tokens"]
        else:
            break
    return tweets

In [122]:
with requests.Session() as session:
    print(get_recent_tweet_ids(session))

[{'id': '1332626878797066240', 'text': '@SoMK (des sets de peinture quelle bonne idée) \n➡️ https://t.co/rLdbPjL20O https://t.co/hT5s5nBMJn'}, {'id': '1332624745754468354', 'text': "@SoMK Pardon j'étais passée à coté mais c'est SUPER !"}, {'id': '1332624289070256144', 'text': '@soyoufr TRES BEAU. Les vends-tu quelque part ?'}, {'id': '1332624048313036801', 'text': "@Layzy_Katsudon @FlamazingS COOL ! Je suis pas futée futée mais j'ai pas trouvé la boutique par contre ?"}, {'id': '1332370064813780995', 'text': '@Tritrichoupi heu hé bah trop beau. \n➡️ https://t.co/NNynOXBk9f https://t.co/qU8k3SAMjN'}, {'id': '1331863293791784960', 'text': '@Layzy_Katsudon Wwoowowowowow. \n\n(Reaction a chaud)'}]
